In [1]:
using Revise
using LinearAlgebra
using SparseArrays
using Pkg

In [13]:
using OpenQuantumSystems

In [14]:
mode1 = Mode(0.2, 1.0)
mode2 = Mode(0.2, 1.0)
Energy = [0.0, 200.0]
mol1 = Molecule([mode1, mode2], 3, [2.0, 200.0])
mol2 = Molecule([mode1], 3, [3.0, 300.0])
aggCore = AggregateCore([mol1, mol2])
agg = Aggregate(aggCore)
agg.core.coupling[2, 3] = 50
agg.core.coupling[3, 2] = 50
agg.tools = AggregateTools(agg)
println("")

In [15]:
agg.operators = AggregateOperators(agg.core, agg.tools)
println("")

In [12]:
agg = setupAggregate(aggCore)

Operator{GenericBasis{Vector{Int64}}, GenericBasis{Vector{Int64}}, Matrix{Float64}}

In [5]:
typeof(indicesMap)

Vector{Vector{Int64}} (alias for Array{Array{Int64, 1}, 1})

In [7]:
indicesMap

81-element Vector{Vector{Vector{T} where T}}:
 [[1, 1], [[1, 1], [1]]]
 [[1, 1], [[1, 1], [2]]]
 [[1, 1], [[1, 1], [3]]]
 [[1, 1], [[1, 2], [1]]]
 [[1, 1], [[1, 2], [2]]]
 [[1, 1], [[1, 2], [3]]]
 [[1, 1], [[1, 3], [1]]]
 [[1, 1], [[1, 3], [2]]]
 [[1, 1], [[1, 3], [3]]]
 [[1, 1], [[2, 1], [1]]]
 [[1, 1], [[2, 1], [2]]]
 [[1, 1], [[2, 1], [3]]]
 [[1, 1], [[2, 2], [1]]]
 ⋮
 [[1, 2], [[2, 3], [1]]]
 [[1, 2], [[2, 3], [2]]]
 [[1, 2], [[2, 3], [3]]]
 [[1, 2], [[3, 1], [1]]]
 [[1, 2], [[3, 1], [2]]]
 [[1, 2], [[3, 1], [3]]]
 [[1, 2], [[3, 2], [1]]]
 [[1, 2], [[3, 2], [2]]]
 [[1, 2], [[3, 2], [3]]]
 [[1, 2], [[3, 3], [1]]]
 [[1, 2], [[3, 3], [2]]]
 [[1, 2], [[3, 3], [3]]]

In [ ]:
typeof(vibindices)

In [8]:
FCfactors = getFranckCondonFactors(aggCore, agg.tools.indices)
typeof(FCfactors) == OpenQuantumSystems.FCfactorsT

Matrix{T} where T<:Integer (alias for Array{T, 2} where T<:Integer)

In [6]:
include("test_molecules.jl")

Test Summary: | Pass  Total
molecules     |   20     20


Test.DefaultTestSet("molecules", Any[], 20, false, false)

In [13]:
const IndicesMap = Vector{Vector{T}} where T <: Integer

┌ Error: Failed to revise c:\Users\daniel.herman\Documents\Mgr\OpenQuantumSystems\OpenQuantumSystems.jl\src\aggregateTools.jl
│   exception = Revise.ReviseEvalException("c:\\Users\\daniel.herman\\Documents\\Mgr\\OpenQuantumSystems\\OpenQuantumSystems.jl\\src\\aggregateTools.jl:4", ErrorException("invalid redefinition of constant ElIndices"), Any[(top-level scope at aggregateTools.jl:4, 1)])
└ @ Revise C:\Users\daniel.herman\.julia\packages\Revise\3RMhb\src\packagedef.jl:714


Array{Vector{T}, 1} where T<:Integer